In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [ ]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [ ]:
def del_time(x):
    original_string = x
    cleaned_string = re.sub(r'\(\d+\)', '', original_string)
    cleaned_string = cleaned_string.strip()
    return cleaned_string


In [5]:
movies_df['title'] =  movies_df.title.apply(del_time)

In [ ]:
movies_df = movies_df.drop('genres', axis=1)

In [7]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [8]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
ratings_df = ratings_df.drop('timestamp', axis=1)

In [10]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [ ]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('year',axis= 1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [ ]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0


In [ ]:
userSubsetGroup = userSubset.groupby(['userId'])

In [ ]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [ ]:
userSubsetGroup[0:3]

[((91,),
         userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14173      91      296     4.5
  14316      91     1274     5.0
  14383      91     1968     3.0),
 ((177,),
         userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  24930     177      296     5.0
  25069     177     1274     2.0
  25129     177     1968     3.5),
 ((219,),
         userId  movieId  rating
  31524     219        1     3.5
  31525     219        2     2.5
  31554     219      296     4.0
  31628     219     1274     2.5
  31680     219     1968     3.0)]

In [17]:
userSubsetGroup = userSubsetGroup[0:100]

In [ ]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    nRatings = len(group)
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [19]:
pearsonCorrelationDict.items()

dict_items([((91,), 0.43852900965351443), ((177,), 0.0), ((219,), 0.45124262819713973), ((274,), 0.716114874039432), ((298,), 0.9592712306918567), ((414,), 0.9376144618769914), ((474,), 0.11720180773462392), ((477,), 0.4385290096535153), ((480,), 0.7844645405527362), ((483,), 0.08006407690254357), ((599,), 0.7666866491579839), ((608,), 0.920736884379251), ((50,), 0.15713484026367722), ((57,), -0.7385489458759964), ((68,), 0.0), ((103,), 0.5222329678670935), ((135,), 0.8703882797784892), ((182,), 0.9428090415820635), ((202,), 0.5222329678670935), ((217,), 0.30151134457776363), ((226,), 0.9438798074485389), ((288,), 0.6005325641789633), ((307,), 0.9655810287305759), ((318,), 0.44486512077567225), ((322,), 0.5057805388588731), ((330,), 0.9035942578600878), ((357,), 0.5606119105813882), ((434,), 0.9864036607532465), ((448,), 0.30151134457776363), ((469,), 0.8164965809277261), ((561,), 0.5222329678670935), ((600,), 0.18442777839082938), ((606,), 0.9146591207600472), ((610,), -0.471404520791

In [20]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.438529,"(91,)"
1,0.000000,"(177,)"
2,0.451243,"(219,)"
3,0.716115,"(274,)"
4,0.959271,"(298,)"


In [21]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
43,1.0,"(132,)"
34,1.0,"(18,)"
63,1.0,"(305,)"
82,1.0,"(489,)"
86,1.0,"(525,)"


In [22]:
topUsers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 43 to 37
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   similarityIndex  50 non-null     float64
 1   userId           50 non-null     object 
dtypes: float64(1), object(1)
memory usage: 1.2+ KB


In [23]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [24]:
ratings_df["userId"]=ratings_df["userId"].astype(object)

In [25]:
ratings_df["userId"].head()

0    1
1    1
2    1
3    1
4    1
Name: userId, dtype: object

In [26]:
topUsers["userId"].head()

43    (132,)
34     (18,)
63    (305,)
82    (489,)
86    (525,)
Name: userId, dtype: object

In [27]:
topUsersRating=pd.concat([topUsers,ratings_df], axis=1, join='inner')
topUsersRating.head()

,similarityIndex,userId,userId,movieId,rating
43,1.0,"(132,)",1,804,4.0
34,1.0,"(18,)",1,593,4.0
63,1.0,"(305,)",1,1090,4.0
82,1.0,"(489,)",1,1256,5.0
86,1.0,"(525,)",1,1275,5.0


In [ ]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,userId,movieId,rating,weightedRating
43,1.0,"(132,)",1,804,4.0,4.0
34,1.0,"(18,)",1,593,4.0,4.0
63,1.0,"(305,)",1,1090,4.0,4.0
82,1.0,"(489,)",1,1256,5.0,5.0
86,1.0,"(525,)",1,1275,5.0,5.0


In [29]:
'''پس از گروه بندی بر اساس
userid
مبلغی برای 
topuser
اعمال میکند'''
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
47,0.716115,3.580574
50,0.959271,4.796356
70,0.937614,2.812843
151,0.784465,3.922323
163,0.766687,3.833433


In [ ]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
47,5.0,47
50,5.0,50
70,3.0,70
151,5.0,151
163,5.0,163


In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
47,5.0,47
50,5.0,50
1298,5.0,1298
1291,5.0,1291
1278,5.0,1278
1275,5.0,1275
1256,5.0,1256
1240,5.0,1240
1222,5.0,1222


In [32]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
46,47,Seven (a.k.a. Se7en),1995
49,50,"Usual Suspects, The",1995
1187,1213,Goodfellas,1990
1196,1222,Full Metal Jacket,1987
1212,1240,"Terminator, The",1984
1228,1256,Duck Soup,1933
1247,1275,Highlander,1986
1250,1278,Young Frankenstein,1974
1263,1291,Indiana Jones and the Last Crusade,1989
1269,1298,Pink Floyd: The Wall,1982
